In [1]:
!pip install transformers datasets
!pip install pytorch_lightning

In [2]:
import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation

from torch.utils.data import TensorDataset, random_split, DataLoader, RandomSampler, SequentialSampler

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl


from transformers import (
    AdamW,
    T5ForSequenceClassification,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)
     

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
df = pd.read_csv("../data/hatebr_and_rationales.csv")

In [4]:
## split to train and val
TRAIN_SIZE = 0.8
TEST_SIZE = 0.1
VAL_SIZE = 0.1
from sklearn.model_selection import train_test_split

x_train, x_test_val, y_train, y_test_val = train_test_split(df['normalized_text'], df['label final'], test_size=TEST_SIZE + VAL_SIZE, random_state=0)
x_test, x_val, y_test, y_val = train_test_split(x_test_val, y_test_val, test_size=VAL_SIZE/(TEST_SIZE + VAL_SIZE), random_state=0)

In [5]:
from torch.utils.data import TensorDataset
import torch
import numpy as np
from transformers import T5Tokenizer

def tokenize_corpus(df, tokenizer, max_len):
    input_ids = []
    attention_masks = []

    for doc in df:
        # Tokenizing input text
        encoded_dict = tokenizer.encode_plus(
                            doc,
                            add_special_tokens=True,
                            max_length=max_len,
                            truncation=True,
                            padding='max_length',
                            return_attention_mask=True,
                            return_tensors='pt'
                       )

        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])

    return torch.cat(input_ids, dim=0), torch.cat(attention_masks, dim=0)

def prepare_dataset(features, labels, tokenizer):
    # Tokenize the input texts
    padded_tokens, attention_masks = tokenize_corpus(features.values, tokenizer, 512) # tokeniza as mensagens
    target = np.array(labels.values, dtype=np.int64).reshape(-1, 1) # transforma target em np array
    tensor_df = TensorDataset(padded_tokens, attention_masks, torch.from_numpy(target))

    return tensor_df


In [6]:
from transformers import T5ForSequenceClassification, T5Tokenizer

# Specify the path or model identifier
model_name_or_path = 'unicamp-dl/ptt5-base-portuguese-vocab'  # Or use a model identifier like 't5-small'

# Load the tokenizer
tokenizer = T5Tokenizer.from_pretrained(model_name_or_path)

# Load the model
model = T5ForSequenceClassification.from_pretrained(model_name_or_path)

spiece.model:   0%|          | 0.00/756k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/456 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at unicamp-dl/ptt5-base-portuguese-vocab and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
# Load the checkpoint
checkpoint = torch.load('../models/ptt5-fine-tuned/best-checkpoint.ckpt', map_location=torch.device('cpu'))

# Extract the state_dict from the checkpoint
state_dict = checkpoint['state_dict']

# Create a new dictionary that matches the expected keys
new_state_dict = {k.replace('model.', ''): v for k, v in state_dict.items()}

# Load the state dictionary into the model
model.load_state_dict(new_state_dict)

/tmp/ipykernel_17/2412763992.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('/kaggle/input/ptt5-fine-tuned/best-checkpoint.ckpt', map_location=t

<All keys matched successfully>

In [8]:
import torch.nn.functional as F

def predict_fn(text):
    model.eval()
    
    padded_tokens, attention_masks = tokenize_corpus(text, tokenizer, 512) # tokeniza as mensagens
    tensor_df = TensorDataset(padded_tokens, attention_masks)
    
    test_dataloader = DataLoader(tensor_df,
                              batch_size=len(text),
                              shuffle=False)
    
    for batch in test_dataloader:
        
        with torch.no_grad():
            outputs = model(input_ids=batch[0],attention_mask= batch[1])
    
    # Get the logits (raw predictions)
    logits = outputs.logits
    
    # Apply softmax to get probabilities
    probs = F.softmax(logits, dim=-1)
    
    # Return probabilities as a numpy array (LIME and SHAP expect numpy arrays)
    return probs.cpu().numpy()

In [9]:
instances = np.where(y_train == 1)[0][0:350]

In [10]:
import shap
class_names = ["Non-hate", "Hate"]
masker = shap.maskers.Text(tokenizer=r"\W+")
explainer = shap.Explainer(predict_fn, masker=masker, output_names=class_names)

shap_values = explainer(x_train.iloc[instances].tolist())

  0%|          | 0/306 [00:00<?, ?it/s]

PartitionExplainer explainer:   1%|          | 1/100 [00:00<?, ?it/s]

  0%|          | 0/498 [00:00<?, ?it/s]

PartitionExplainer explainer:   3%|▎         | 3/100 [37:32<20:23:35, 756.86s/it]

  0%|          | 0/2 [00:00<?, ?it/s]

...

PartitionExplainer explainer:  99%|█████████▉| 99/100 [9:30:23<06:17, 377.05s/it]

  0%|          | 0/12 [00:00<?, ?it/s]

PartitionExplainer explainer: 100%|██████████| 100/100 [9:30:53<00:00, 272.83s/it]

  0%|          | 0/20 [00:00<?, ?it/s]

PartitionExplainer explainer: 101it [9:31:42, 343.03s/it]


In [11]:
with open('../results/results_shap_ptt5_0_a_350.txt', 'w') as f:
    for i in range(len(instances)):
        f.write('[')
        for j, (word, score) in enumerate(zip(shap_values[i,:, class_names[1]].data, shap_values[i,:, class_names[1]].values)):
            f.write(str((word, score)))    
            if j < len(shap_values[i,:, class_names[1]].data)-1:
                f.write(', ')
        f.write(']\n')